# <p style="color:dodgerblue">Image Parsing in Bedrock</p>
<hr style="border:1px dotted; color:floralwhite">

# Requirements for this lab
*See <span style="color:gold">Appendix</span> at the bottom of this lab to install requirements for this lab*

<hr style="border:1px dotted">
<hr style="border:1px dotted;color:cadetblue">

# <p style="color:cadetblue">Import libraries and set clients</p>
*If the following does not show the latest (installed or upgraded) boto3 (at least v1.29.6), restart the kernel*

In [38]:
import boto3, json
import base64
import pprint

# region_name is important especially if you auth via aws configure and it is set to a region where there is no Bedrock yet
#define common vars
myRegion='us-west-2'

# get clients
bedrockRun = boto3.client(service_name='bedrock-runtime', region_name=myRegion)
pp = pprint.PrettyPrinter(indent=4)

boto3.__version__

# local client path for resources
myLocalPathForDataSources='/Users/simondavies/Documents/GitHub/labs/bedrock/quick/Resources/'

# jupypter notebook path if notebook is used in AWS for example
#myLocalPathForDataSources='/home/ec2-user/SageMaker/labs/bedrock/quick/Resources/'

<hr style="border:1px dotted;color:cadetblue">
<hr style="border:1px dotted;color:crimson">

# <p style="color:crimson">Create some methods to call</p>
1. Method for processing a prompt
2. Method to have a conversation
3. <b>Don't forget if you modify this method, make sure you execute the cell</b>

### <p style="color:crimson">This method processes a single prompt with the chosen foundation model</p>

In [40]:
# open and encode an image
def encodeImage(imagePath): 
    with open(imagePath, "rb") as imageFile: 
        return base64.b64encode(imageFile.read()).decode("utf-8")

In [41]:
# processes a single prompt for image OCR
def processOCR(m, prompt, p, t, k, n, base64Image):
    # prep some vars
    body = ""
    outputResponse = "Model: {}\n---------------------------------------\n".format(m)
    accept = "application/json"
    contentType = "application/json"
    tk = 2000

    # which model are we calling
    match m:
        case "anthropic.claude-3-haiku-20240307-v1:0":
            # Anthropic - Claude 3 Haiku
            # https://www.anthropic.com/news/claude-3-haiku
            body = json.dumps(
                {
                    "anthropic_version": "bedrock-2023-05-31",
                    "max_tokens": tk,
                    "messages": [
                        {
                            "role": "user",
                            "content": [
                                {
                                    "type": "image",
                                    "source": {
                                        "type": "base64",
                                        "media_type": "image/png",
                                        "data": base64Image,
                                    },
                                },
                                {"type": "text", "text": prompt},
                            ],
                        }
                    ],
                    "temperature": t,
                    "top_p": p,
                    "top_k": k,
                }
            )

    # invoke the model
    response = bedrockRun.invoke_model(
        body=body, modelId=m, accept=accept, contentType=contentType
    )
    responseBody = json.loads(response.get("body").read())

    # get the response
    match m:
        case "anthropic.claude-3-haiku-20240307-v1:0":
            outputText = responseBody["content"]
            outputResponse += "1 RESPONSE Supported:\n{}\n---------------------------------------\n".format(
                outputText
            )

    # return response
    return outputResponse

# <p style="color:crimson">Now we can start using our models with prompts</p>

<hr style="border:1px dotted;color:crimson">
<hr style="border:1px dotted;color:lightblue">

# <p style="color:lightblue">Perform OCR on an image</p>
1. Specify properties
2. Call the method
3. Prompt engineering <a href='https://docs.aws.amazon.com/bedrock/latest/userguide/general-guidelines-for-bedrock-users.html'>help</a>

In [45]:
# set up the prompt and params - note each model will have different ranges for its params
prompt="""Your purpose is to analyze invoices.
            Create a structured set of data in json format providing key information about an invoice.   
            Do not return any narrative language. 
            JSON fields must be labelled as:  
            - INVOICE_NUMBER,  
            - CUSTOMER_NUMBER,  
            - CUSTOMER_NAME,  
            - VENDOR_NAME,  
            - INVOICE_AMOUNT,  
            - INVOICE_DATE (show in this format YYYY-MM-DD),  
            - VENDOR_ADDRESS.  
            Example json structure is: 
            <json> 
                { 
                    “INVOICE_NUMBER”: “Invoice number”, 
                    “CUSTOMER_NUMBER”: “Customer Number”, 
                    “CUSTOMER_NAME”: “Customer Name”, 
                    “VENDOR_NAME”: “Vendor Name”, 
                    “INVOICE_AMOUNT”: “Invoice Amount”, 
                    “INVOICE_DATE”: “Invoice Date”, 
                    “VENDOR_ADDRESS”: “Vendor Address” 
                } 
            </json>              

            Output the json structure as a string starting with <json> and ending with </json> XML tags.   
            Do not return any narrative language. Look at the images in detail, looking for invoice number,  
            customer number, customer name, vendor name,  
            invoice amount, invoice date, and vendor address where possible try to identify them. 
            IF YOU DO NOT FIND ANY AMOUNT, ROTATE THE IMAGE CLOCKWISE AND TRY AGAIN.
            IF YOU COULD NOT FIND THE RIGHT INFORMATION JUST RETURN THIS VALUE 'UNKNOWN'."""


t=0.7 # temperature: use a lower value to decrease randomness in the response
p=0.5 # topP: use a lower value to ignore less probable options
k=50 # topK: number of most-likely candidates that the model considers for the next token
n=1 # numResults: how many responses do you want, not all models support multiple responses

# encode the image into base 64
base64Image = encodeImage('{}{}'.format(myLocalPathForDataSources,'receipt rotated.png'))

# which model do you want to use based on the match statement below
u=1

match u:
    case 1:
        # Anthropic - Claude 3 Haiku
        # property ranges: https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html
        m='anthropic.claude-3-haiku-20240307-v1:0'

o=processOCR(m, prompt, p, t, k, n, base64Image)
print(o)
# pp.pprint(str(o)) 

Model: anthropic.claude-3-haiku-20240307-v1:0
---------------------------------------
1 RESPONSE Supported:
[{'type': 'text', 'text': '<json>\n{\n    "INVOICE_NUMBER": "1.1",\n    "CUSTOMER_NUMBER": "UNKNOWN",\n    "CUSTOMER_NAME": "UNKNOWN",\n    "VENDOR_NAME": "UNKNOWN",\n    "INVOICE_AMOUNT": "1.30",\n    "INVOICE_DATE": "2024-01-07",\n    "VENDOR_ADDRESS": "UNKNOWN"\n}\n</json>'}]
---------------------------------------



<hr style="border:1px dotted;color:lightblue">
<hr style="border:1px dotted;color:deeppink">


<hr style="border:1px dotted;color:coral">
<hr style="border:1px dotted;color:gold">

# <p style="color:gold">Appendix - Install Requirements (macOS)</p>
# Requirements for this lab

#### <p style="color:deeppink">- If you are running VSCode on a laptop, follow all of below.<br>- If you are running Jupyter inside an AWS Account, you just need to install langchain. See 2.1 below and skip everything else.</p>

*Windows requirements will be similar, apart from Homebrew.*  
* python must be installed on your client, and be at least v3.8
  * 3.8 supports the latest release of boto3 which supports Bedrock  
* boto3 must be installed on your client, and be at least v1.33.9  
  * *Boto3 is the Amazon Web Services (AWS) Software Development Kit (SDK) for Python, which allows Python developers to write software that makes use of services like Amazon S3 and Amazon EC2.*
  * https://boto3.amazonaws.com/v1/documentation/api/latest/index.html
### <p style="color:gold">1. Homebrew</p> 
If you haven't installed Homebrew, you can install it by running the following command here or in the terminal:

In [ ]:
%%bash
sudo /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"

### <p style="color:gold">1. Python</p> 
Once Homebrew is installed, you can install Python using the following command  
*check what you have before installing/upgrading*  
*you will need to quit and restart vsCode to use python once installed (or updated)*

In [ ]:
%%bash
python3 --version
which python3

In [ ]:
%%bash
brew install python

### <p style="color:gold">2. boto3 and other Python requirements</p> 
*check what you have before installing/upgrading*  

In [ ]:
%%bash
python3 -m pip show boto3

In [ ]:
pip install -U boto3

### <p style="color:gold">2.1 Chatbot requirements</p> 
*<p style="color:deeppink">If you are running a Jupyter in AWS Account, you ONLY need these 2 cells</p>*

In [ ]:
pip install -U langchain

In [ ]:
pip install langchain-community langchain-core

### <p style="color:gold">3. aws configure</p> 
*Configure aws configure with credentials, and a user that has all of the Bedrock IAM policies required*  
https://docs.aws.amazon.com/bedrock/latest/userguide/security_iam_id-based-policy-examples.html

In [ ]:
%%bash
aws sts get-caller-identity

### <p style="color:gold">4. Request Bedrock model access</p> 
*You must request access to the models required, you may need to provide use case details before you are able to request*  
*Make sure you request in the region you intend to use the models in, this lab is us-west-2*  
https://us-west-2.console.aws.amazon.com/bedrock/home?region=us-west-2#/modelaccess  

Models required in this lab:

* See code above for use of models and what access to request

#### Pricing
*Use 6 characters per token as an approximation for the number of tokens.*  
https://aws.amazon.com/bedrock/pricing/  
https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-prepare.html

<hr style="border:1px dotted;color:gold">